In [7]:
import sys
sys.path.append('./base_model')

In [8]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import umap
from interface import PolyDisVAE
from base_model import converter

### Configurações

In [9]:
FEATURE_DIR = 'features'
MODEL_PATH = 'model_param/polydis-v1.pt'
BATCH_SIZE = 64
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
from model import DisentangleVAE

In [10]:
model = PolyDisVAE.init_model(device=DEVICE)
if os.path.exists(MODEL_PATH):
    model.load_model(MODEL_PATH)
model.eval()

PolyDisVAE(
  (chd_encoder): ChordEncoder(
    (gru): GRU(36, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (txt_encoder): TextureEncoder(
    (cnn): Sequential(
      (0): Conv2d(1, 10, kernel_size=(4, 12), stride=(4, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Linear(in_features=290, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=256, bias=True)
    (gru): GRU(256, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (pnotree_decoder): PianoTreeDecoder(
    (note_embedding): Linear(in_features=135, out_features=128, bias=True)
    (z2dec_hid_linear): Linear(in_features=512, out_fea

In [11]:
def batch_iterator(feature_dir, batch_size):
    files = sorted([f for f in os.listdir(feature_dir) if f.endswith('.npz')])
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i+batch_size]
        piano_rolls = []
        chords = []
        for bf in batch_files:
            data = np.load(os.path.join(feature_dir, bf))
            piano_rolls.append(data['piano_roll'])
            chords.append(data['chord'])
        yield np.stack(piano_rolls), np.stack(chords)

In [14]:
latents_txt = []
latents_chd = []
recon_losses = []

for pr_batch, chd_batch in batch_iterator(FEATURE_DIR, BATCH_SIZE):
    x = np.stack([converter.target_to_3dtarget(pr) for pr in pr_batch])
    pr_t = torch.tensor(pr_batch, dtype=torch.float32, device=DEVICE)
    chd_t = torch.tensor(chd_batch, dtype=torch.float32, device=DEVICE)
    x_t = torch.tensor(x, dtype=torch.long, device=DEVICE)

    with torch.no_grad():
        out = DisentangleVAE.loss(x_t, chd_t, pr_t)
        dist_chd, dist_txt = model.inference_encode(pr_t, chd_t)
    latents_chd.append(dist_chd.mean.cpu().numpy())
    latents_txt.append(dist_txt.mean.cpu().numpy())
    recon_losses.append(out[1].cpu().numpy())

latents_txt = np.concatenate(latents_txt)
latents_chd = np.concatenate(latents_chd)
recon_losses = np.concatenate(recon_losses)

AttributeError: 'PolyDisVAE' object has no attribute 'loss'

In [ ]:
reducer = umap.UMAP()
txt_2d = reducer.fit_transform(latents_txt)
chd_2d = reducer.fit_transform(latents_chd)

fig, axs = plt.subplots(1, 2, figsize=(12,5))
sc0 = axs[0].scatter(txt_2d[:,0], txt_2d[:,1], c=recon_losses, cmap='viridis')
axs[0].set_title('Textura (UMAP)')
plt.colorbar(sc0, ax=axs[0])
sc1 = axs[1].scatter(chd_2d[:,0], chd_2d[:,1], c=recon_losses, cmap='viridis')
axs[1].set_title('Acorde (UMAP)')
plt.colorbar(sc1, ax=axs[1])
plt.show()